# Support vector machine
Tentativo di fare la previsione con una support vector machine

In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
#pipelines for scaling data
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#setting up labels for dataset
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
#display(df)
#separating y from x
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

In [2]:
#declaring classifier for 'one versus one' case
clf = svm.SVC(decision_function_shape='ovo')
print(type(clf))

<class 'sklearn.svm._classes.SVC'>


In [3]:
#fitting classifier
clf.fit(X, y)

SVC(decision_function_shape='ovo')

## Static test

In [4]:
#dividing the dataset for a static test
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 2:16], df.iloc[:, 0], test_size=0.3) # 70% training and 30% test

Nota: se vuoi ottenere sempre lo stesso split (ad esempio per fare delle prove), aggiungi il parametro "random_state = num" dove num è un intero costante
Nota: to_frame() è un metodo delle variabili di tipo "Series" per convertirle al tipo DataFrame. La "train_test_split", infatti, mi restituisce DataFrame per le X e Series per le Y, visto che quest'ultime sono colonne singole. Le funzioni successive non funzionano con parametri di tipo Series, quindi è necessario aggiungere to_frame(). Altre info qua: https://stackoverflow.com/questions/26047209/what-is-the-difference-between-a-pandas-series-and-a-single-column-dataframe 

In [5]:
scaled_clf = make_pipeline(StandardScaler(), svm.SVC(kernel='linear'))
scaled_clf.fit(X_train, y_train)

stat_clf = svm.SVC(kernel='linear')    #remember that the default kernel is rbf. 
#Try changing kernel between: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} 
print(type(y_train))
stat_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])

<class 'pandas.core.series.Series'>


SVC(kernel='linear')

In [6]:
#let's make a prediction on the test data with both classifiers
y_pred = stat_clf.predict(X_test)
y_pred_scaled = scaled_clf.predict(X_test)
y_pred - y_pred_scaled

array([  0, -20,   0, -20,  -3,   0,   0,  -2,   0,   0,   0,   2,   0,
         0,   0,   0, -25,   0, -20,   2,   0,   2,  28,   0,   0,  14,
         0,   0,  17,   0,  -6,   2,   0, -25, -20,  21,   0,  30,   3,
         0,  16,   0,   0,   0,   0,   0,   0,   0,  28,   2,   0, -21,
         0,  10,   0,   0,   5, -25,  27,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, -16,   6,  14,   0,   0,   0,   0,   0,   0,
       -15,  14,  30,   0,  14,  21,  16,   0,  -5,   0,   2,  11,   0,
         0,   6,  -7,   0,   2,   0,   5,   0,  14,  -3,   0])

Possiamo vedere che le previsioni sono diverse. Proviamo a confrontare i parametri di stat_clf e scaled_clf, per vedere se con lo scaling cambiano solo i dati o anche i classificatori.

### Secondo me
stat_clf e what_is_this sono probabilmente lo stesso classifier SVM, perchè hanno sia lo stesso "type" che le stesse previsioni nel test set.

### Valutiamo il nostro modello statico

In [7]:
print("Accuracy clf:", metrics.accuracy_score(y_test, y_pred))
print("Accuracy scaled_clf:", metrics.accuracy_score(y_test, y_pred_scaled))

Accuracy clf: 0.5
Accuracy scaled_clf: 0.7352941176470589


## Cross validation
Facciamo learning tramite cross validation su un terzo modello "clf_cv"

In [8]:
#print(metrics.get_scorer_names())

In [9]:
clf_cv = svm.SVC(decision_function_shape='ovo')
k = 5
effect_cv = cross_validate(clf_cv, X, y, cv=k, scoring=('accuracy'))
effect_cv
print("Mean test_score:", mean(effect_cv['test_score']))

{'fit_time': array([0.00725961, 0.00670195, 0.00637317, 0.0058701 , 0.00609756]),
 'score_time': array([0.00289249, 0.00221014, 0.00228024, 0.001791  , 0.00218987]),
 'test_score': array([0.29411765, 0.30882353, 0.27941176, 0.27941176, 0.29411765])}

Mean test_score: 0.2911764705882353


In [10]:
#pipelines for scaling data
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([('scaling', StandardScaler()),
                 ('SVM', svm.SVC(decision_function_shape='ovo'))])

# building the range of the regularization parameter (C)
exp = np.arange(-10, 12)
reg_param = 10.**exp

grid_param = {'SVM__C': reg_param,
              'SVM__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'SVM__degree': np.arange(2, 5),
              'SVM__decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(pipe, grid_param, n_jobs=2)
clf_cv.fit(X,y)
print("Best params:", clf_cv.best_params_)

GridSearchCV(estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('SVM',
                                        SVC(decision_function_shape='ovo'))]),
             n_jobs=2,
             param_grid={'SVM__C': array([1.e-10, 1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03,
       1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10, 1.e+11]),
                         'SVM__decision_function_shape': ['ovo', 'ovr'],
                         'SVM__degree': array([2, 3, 4]),
                         'SVM__kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

Best params: {'SVM__C': 100.0, 'SVM__decision_function_shape': 'ovo', 'SVM__degree': 2, 'SVM__kernel': 'linear'}


**Io penso che nel blocco precedente manchi la parte in cui lo scaler viene fittato sulla k-fold**